# Scrapeo de las competiciones de salto

In [2]:
# Importamos las librerías necesarias
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
from selenium.webdriver.support.ui import WebDriverWait
# import xlrd
# from src.soporte_scrapeo_general import get_competiciones, cambio_pestaña, resultados_disciplina, buscador_elementos, guardado_info, competiciones_año, obtencion_año, creacion_dictios_guardado
# from src.soporte_scrapeo_salto import extraccion_resultados_jinetes_caballos

In [2]:
# URL inicial
url = "https://rfhe.com/competiciones/"

In [3]:
# inicializamos el driver y lo abrimos
driver = get_competiciones(url)

In [4]:
# creamos los diccionarios en los que vamos a ir guardando toda la información
dictio_concursos, dictio_pruebas, dictio_jinetes, dictio_caballos = creacion_dictios_guardado()

In [5]:
# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, disciplina = "salto")

In [6]:
ambito_buscado, disciplina_buscada

('nacional', 'salto')

In [7]:
# Buscamos los concursos de febrero del año que nos aparece
competiciones_año(driver)

In [8]:
# Sacamos el total de concursos que hay en el mes de marzo, que compondrá el total de veces que tendrá que iterar el bucle for siguiente
maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[2]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))
maximo_n_concursos


40

In [9]:
# Buscamos el primer concurso del mes de marzo
i = 5
selector = f"/html/body/form/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[4]/a"
concurso = buscador_elementos(driver, selector).click()

In [10]:
cambio_pestaña(2,driver)

In [11]:
# Sacamos el contenido general del concurso
contenido_general =  buscador_elementos(driver, "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
contenido_general

['Nombre: CSN2* RACE',
 'Categoría: CSN**',
 'Provincia: Madrid',
 'Localidad: R.A.C.E.',
 'Disciplina: Salto de Obstáculos',
 'Fechas: 01 al 02 de Febrero de 2025',
 'Federación: Federación Hípica de Madrid',
 'Avance: Disponible en PDF  ',
 '  Fechas incripción: 15/Ene->27/Ene/2025',
 'Inscripción: Concurso Finalizado',
 'Admitidos: Publicada  Ver',
 'Resultados: Ver resultados  Ver']

In [12]:
# Guardamos la información del concurso en el diccionario creado
guardado_info(dictio_concursos, contenido_general, info = "concursos", ambito = ambito_buscado)

In [13]:
dictio_concursos

{'Nombre': ['CSN2* RACE'],
 'Categoría': ['CSN**'],
 'Provincia': ['Madrid'],
 'Localidad': ['R.A.C.E.'],
 'Disciplina': ['Salto de Obstáculos'],
 'Federación': ['Federación Hípica de Madrid'],
 'Resultados': ['Ver'],
 'País': ['España']}

In [14]:
# Nos vamos a las pruebas del concurso
path_boton_pruebas = "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody/tr[20]/td[3]/a"
pruebas = buscador_elementos(driver, path_boton_pruebas).click()
cambio_pestaña(3,driver)

In [15]:
# Obtenemos la información del concurso que se encuentra en la tabla de las pruebas
path_info_restante = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td"
info_concurso_restante = buscador_elementos(driver, path_info_restante).text.split("\n")

# Creamos las nuevas claves que sacamos de la info que esta donde las pruebas
dictio_concursos[info_concurso_restante[2].strip(":")] = []
dictio_concursos[info_concurso_restante[4].strip(":")] = []
dictio_concursos[info_concurso_restante[8].strip(":")] = []

In [16]:
# metemos la info del concurso que nos falta
guardado_info(diccionario = dictio_concursos, elementos = info_concurso_restante, claves = ["Inicio", "Final", "Ámbito"], indices = [3, 5, -1], info = "general")

In [17]:
dictio_concursos

{'Nombre': ['CSN2* RACE'],
 'Categoría': ['CSN**'],
 'Provincia': ['Madrid'],
 'Localidad': ['R.A.C.E.'],
 'Disciplina': ['Salto de Obstáculos'],
 'Federación': ['Federación Hípica de Madrid'],
 'Resultados': ['Ver'],
 'País': ['España'],
 'Inicio': ['01/02/2025'],
 'Final': ['02/02/2025'],
 'Ámbito': ['Nacional']}

In [18]:
# Obtenemos el nombre del concurso para luego meterlo en la tabla de pruebas
path_nombre_concurso = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td/div/div[1]/div/table/tbody/tr/td[2]/table/tbody/tr"
nombre_concurso = buscador_elementos(driver, path_nombre_concurso).text.strip()
nombre_concurso

'CSN2* RACE'

In [19]:
# Obtenemos las pruebas del concurso
path_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]"
pruebas = buscador_elementos(driver, path_pruebas).text.split("\n")
pruebas

['Disciplina',
 'Fecha',
 'Prueba',
 'Categoría',
 'Número',
 'Resultados',
 'Salto de Obstáculos',
 '01/02/2025',
 '1,10 - Acc (238.2.1)',
 'CSN**',
 '1',
 'Disponibles',
 'Salto de Obstáculos',
 '01/02/2025',
 '1,20 - A c.c. (238.2.1)',
 'CSN**',
 '2',
 'Disponibles',
 'Salto de Obstáculos',
 '01/02/2025',
 '1,30 - A c.c. (238.2.1)',
 'CSN**',
 '3',
 'Disponibles',
 'Salto de Obstáculos',
 '02/02/2025',
 '1,25 - A c.c. (238.2.1)',
 'CSN**',
 '4',
 'Disponibles',
 'Salto de Obstáculos',
 '02/02/2025',
 '1,35 GP - A.c.c. y desempate (238.2.2)',
 'CSN**',
 '5',
 'Disponibles',
 'Salto de Obstáculos',
 '02/02/2025',
 '1,15 - A c.c. (238.2.1)',
 'CSN**',
 '6',
 'Disponibles']

In [ ]:
claves_pruebas = list(dictio_pruebas.keys())
guardado_info(diccionario = dictio_pruebas,elementos = pruebas, claves = claves_pruebas, indices = [6, 7, 8, 9, 10], step = 6, guardado = False, info = "general")

numero_pruebas = len(pruebas[6::6])  # Asumimos que 'Disciplina' tiene una fila por prueba
dictio_pruebas["Concurso"].extend([nombre_concurso] * numero_pruebas)

In [20]:
# accedemos a los resultados de la primera prueba
path_resultados_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]/div[2]/div/table/tbody/tr/td/div/div/table/tbody/tr[1]/td/div/div/table/tbody/tr[1]/td/div/div[3]/div/table/tbody/tr/td/a"
resultados_pruebas = buscador_elementos(driver, path_resultados_pruebas).click()

In [ ]:
extraccion_resultados_jinetes_caballos(driver, dictio_jinetes, dictio_caballos)

In [ ]:
url = "https://rfhe.com/competiciones/"

# inicializamos el driver y lo abrimos
driver = get_competiciones(url)

# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y salto
ambito_buscado, disciplina_buscada = resultados_disciplina(driver)

# Como queremos iterar desde el año 2025, el cual nos aparece de manera automática al abrir la página, hasta 2017, creamos un bucle el cual 
# va iterando por los años hasta 2017, que es el último año del cual queremos obtener datos.

año = obtencion_año(driver)

# vamos a buscar solo las competiciones antes de 2017
while año > 2017:

    # Buscamos los concursos de marzo del año que nos aparece
    competiciones_año(driver)

    # Sacamos el total de concursos que hay en el mes de marzo, que compondrá el total de veces que tendrá que iterar el bucle for siguiente
    #maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[3]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))


    #for i in range (5, maximo_n_concursos + 5)
    for i in range (5, 7):

        try:
            
            selector = f"/html/body/form/div/div/div/div/div[3]/table/tbody/tr[{i}]/td[4]/a"
            concurso = buscador_elementos(driver, selector).click()
            cambio_pestaña(2,driver)

            time.sleep(5)

            # Sacamos el contenido general del concurso
            contenido_general =  buscador_elementos(driver, "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody").text.split('\n')

            if i == 5: 

                elementos = [elemento.split(":")[0] for elemento in contenido_general]

                indices_a_excluir = [5, 7, 8, 9, 10]

                lista_filtrada = [valor for i, valor in enumerate(elementos) if i not in indices_a_excluir]

                dictio_concursos = {entrada: [] for entrada in lista_filtrada} 
                dictio_concursos["País"] = [] 

                # Guardamos la información del concurso en el diccionario creado
                guardado_info(dictio_concursos, contenido_general, info = "concursos", ambito = ambito_buscado)

                # Nos vamos a las pruebas del concurso
                path_boton_pruebas = "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody/tr[20]/td[3]/a"
                pruebas = buscador_elementos(driver, path_boton_pruebas).click()
                cambio_pestaña(3,driver)

                time.sleep(5)

                # Obtenemos la información del concurso que se encuentra en la tabla de las pruebas
                path_info_restante = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td"
                info_concurso_restante = buscador_elementos(driver, path_info_restante).text.split("\n")

                # Creamos las nuevas claves que sacamos de la info que esta donde las pruebas
                dictio_concursos[info_concurso_restante[2].strip(":")] = []
                dictio_concursos[info_concurso_restante[4].strip(":")] = []
                dictio_concursos[info_concurso_restante[8].strip(":")] = []

                # metemos la info del concurso que nos falta
                guardado_info(diccionario = dictio_concursos, elementos = info_concurso_restante, claves = ["Inicio", "Final", "Ámbito"], indices = [3, 5, -1], info = "general")
                
                # Obtenemos el nombre del concurso para luego meterlo en la tabla de pruebas
                path_nombre_concurso = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td/div/div[1]/div/table/tbody/tr/td[2]/table/tbody/tr"
                nombre_concurso = buscador_elementos(driver, path_nombre_concurso).text.strip()

                # pruebas del concurso
                path_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]"
                pruebas = buscador_elementos(driver, path_pruebas).text.split("\n")

                # Creamos el diccionario en el que guardaremos las pruebas
                diccionario_pruebas = {encabezado_prueba : [] for encabezado_prueba in pruebas[:5]}
                diccionario_pruebas["Concurso"] = []

                claves_pruebas = ["Disciplina", "Fecha", "Prueba", "Categoría", "Número"]
                guardado_info(diccionario = diccionario_pruebas,elementos = pruebas, claves = claves_pruebas, indices = [6, 7, 8, 9, 10], step = 6, guardado = False, info = "general")

                numero_pruebas = len(pruebas[6::6])  # Asumimos que 'Disciplina' tiene una fila por prueba
                diccionario_pruebas["Concurso"].extend([nombre_concurso] * numero_pruebas)

                dictio_jinetes = {"Nombre": [],
                                    "Licencia": [],
                                    "Sexo": [],
                                    "País": [],
                                    "Federación": [],
                                    "Disciplina": []}
                dictio_caballos = {"Nombre": [],
                                    "Licencia": [],
                                    "Edad": [],
                                    "País": [],
                                    "Raza": [],
                                    "Sexo": [],
                                    "Federación": [],
                                    "Disciplina": []}

                # accedemos a los resultados de la primera prueba
                path_resultados_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]/div[2]/div/table/tbody/tr/td/div/div/table/tbody/tr[1]/td/div/div/table/tbody/tr[1]/td/div/div[3]/div/table/tbody/tr/td/a"
                resultados_pruebas = buscador_elementos(driver, path_resultados_pruebas).click()

                # Obtenemos todos los resultados
                resultados_prueba = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]/div/table"
                resultados = buscador_elementos(driver, resultados_prueba).text.split("\n")

                # creamos el diccionario en el que iremos guardando los resultados 
                encabezados_malos = resultados[0:8]
                encabezados = [encabezados_malos[0], 
                                encabezados_malos[1].split(' ')[0], 
                                encabezados_malos[4] + 'jinete', 
                                encabezados_malos[1].split(' ')[1], 
                                encabezados_malos[6] + 'caballo',
                                encabezados_malos[2],
                                encabezados_malos[2]+ '2',
                                encabezados_malos[3]]
                                            
                # Transformamos la lista en un diccionario vacío mediante el método fromkeys, el cual contendrá como claves los elementos de la lista creada antes, y 
                # los valores estarán vacíos, ya que los añadiremos a continuación desde los resultados obtenidos. 
                diccionario_resultados = {encabezado: [] for encabezado in encabezados}
                diccionario_resultados["Prueba"] = []

                siguiente_prueba = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[2]/div[2]/div[6]/div/table/tbody/tr/td/a"
                jinetes_caballos(driver)   
                        
                while True:
                    try:
                        buscador_elementos(driver, siguiente_prueba).click()

                        WebDriverWait(driver, 5).until(EC.alert_is_present())
                        alert = driver.switch_to.alert
                        print("alert Exists in page")

                        alert.accept()         
                        driver.close()
                        cambio_pestaña(2,driver)
                        break
                                
                    except TimeoutException:
                        print("alert does not Exist in page")

                        buscador_elementos(driver, siguiente_prueba).click()

                        extraccion_resultados(driver, diccionario_resultados = diccionario_resultados)
                        jinetes_caballos(driver, dictio_jinetes, dictio_caballos)
                    
                    except UnexpectedAlertPresentException:
                        # En caso de que la alerta se dispare justo después del click (por si se salta el `until`)
                        alert = driver.switch_to.alert
                        print("Alerta inesperada después del clic:", alert.text)
                        alert.accept()
                        driver.close()
                        cambio_pestaña(2, driver)
                        break

                time.sleep(5)

                driver.close()
                cambio_pestaña(1, driver)

            elif i != 5 and "Resultados: Ver resultados  Ver" in contenido_general:
                
                # Guardamos la información del concurso en el diccionario creado
                guardado_info(dictio_concursos, contenido_general, info = "concursos", ambito = ambito_buscado)
                    
                # Accedemos a las pruebas del concurso
                try:
                    # Nos vamos a las pruebas del concurso
                    path_boton_pruebas = "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody/tr[20]/td[3]/a"
                    pruebas = buscador_elementos(driver, path_boton_pruebas).click()

                except NoSuchElementException:
                    path_boton_pruebas = "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table/tbody/tr[18]/td[3]/a"
                    pruebas = buscador_elementos(driver, path_boton_pruebas).click()
                
                # Nos vamos a la pestaña con las pruebas
                cambio_pestaña(3,driver)    

                time.sleep(5)

                # Obtenemos la información del concurso que se encuentra en la tabla de las pruebas
                path_info_restante = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td"
                info_concurso_restante = buscador_elementos(driver, path_info_restante).text.split("\n")

                # metemos la info del concurso que nos falta
                guardado_info(diccionario = dictio_concursos, elementos = info_concurso_restante, claves = ["Inicio", "Final", "Ámbito"], indices = [3, 5, -1], info = "general")

                # Obtenemos el nombre del concurso para luego meterlo en la tabla de pruebas
                path_nombre_concurso = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr/td/div/div[1]/div/table/tbody/tr/td[2]/table/tbody/tr"
                nombre_concurso = buscador_elementos(driver, path_nombre_concurso).text.strip()

                # pruebas del concurso
                path_pruebas = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[2]"
                pruebas = buscador_elementos(driver, path_pruebas).text.split("\n")
                    
                claves_pruebas = ["Disciplina", "Fecha", "Prueba", "Categoría", "Número"]
                guardado_info(diccionario = diccionario_pruebas,elementos = pruebas, claves = claves_pruebas, indices = [6, 7, 8, 9, 10], step = 6, guardado = False, info = "general")

                numero_pruebas = len(pruebas[6::6])  # Asumimos que 'Disciplina' tiene una fila por prueba
                diccionario_pruebas["Concurso"].extend([nombre_concurso] * numero_pruebas)
                
                siguiente_prueba = "/html/body/form/table/tbody/tr/td/div/div/table/tbody/tr/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td/div/div/div[1]/div[2]/div[2]/div[6]/div/table/tbody/tr/td/a"
                jinetes_caballos(driver)   
                        
                while True:
                    try:
                        buscador_elementos(driver, siguiente_prueba).click()

                        WebDriverWait(driver, 5).until(EC.alert_is_present())
                        alert = driver.switch_to.alert
                        print("alert Exists in page")

                        alert.accept()         
                        driver.close()
                        cambio_pestaña(2,driver)
                        break
                                
                    except TimeoutException:
                        print("alert does not Exist in page")

                        buscador_elementos(driver, siguiente_prueba).click()

                        extraccion_resultados(driver, diccionario_resultados = diccionario_resultados)
                        jinetes_caballos(driver, dictio_jinetes, dictio_caballos)
                    
                    except UnexpectedAlertPresentException:
                        # En caso de que la alerta se dispare justo después del click (por si se salta el `until`)
                        alert = driver.switch_to.alert
                        print("Alerta inesperada después del clic:", alert.text)
                        alert.accept()
                        driver.close()
                        cambio_pestaña(2, driver)
                        break

                time.sleep(5)

                driver.close()
                cambio_pestaña(1, driver)
                
            # aqui tenemos los concursos que han sido suspendidos
            elif i != 5 and "Resultados: Ver resultados  Ver" not in contenido_general:
                continue

        # Manejamos las competiciones que han sido aplazadas, las cuales no tendremos en cuenta
        except NoSuchElementException:

            enlace_concurso = driver.find_element(By.XPATH, f"/html/body/form/div/div/div/div/div[3]/table/tbody/tr[{i}]/td[4]/font")
            print(enlace_concurso.text)
            continue

        except NoSuchElementException:
            driver.refresh()
            continue

    buscador_elementos(driver,"/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[2]/a").click()

    año = obtencion_año(driver)

alert does not Exist in page


KeyError: 'Lic.Jinete'